In [ ]:
# import libraries
import ee
import pandas as pd
import geemap
import geopandas as gpd
import leafmap

In [ ]:
# initialize the Earth Engine module
ee.Authenticate()
ee.Initialize(project="ee-forestplotvariables")

In [ ]:
# use geopandas to read csv and convert to geodataframe or geojson then convert to ee FeatureCollection

In [ ]:
# coordinates for the points of interest
point = ee.Geometry.Point([77.54849920033682, 12.91215102400037])
fc = ee.FeatureCollection(point)

In [ ]:
terraclimate = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")
tmax = terraclimate.select("tmmx")


def scale_image(image):
    return image.multiply(0.1).copyProperties(image, ["system:time_start"])


tmax_scaled = tmax.map(scale_image)

In [ ]:
m = geemap.Map()
m.add_layer(filtered.first())
m.addLayer(fc, {}, "Points")
m

In [ ]:
start_date = "1990-01-01"
end_date = "2020-12-31"
filtered = tmax_scaled.filterDate(start_date, end_date)
filtered.size().getInfo()
filtered.first().getInfo()
geemap.zonal_stats(filtered.toBands(), fc, scale=1000, statistic_type="MEAN")

# probably don't need anything under this

In [ ]:
def extract_values(image):
    return image.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=4638.3,  # TerraClimate resolution
    )


extracted = filtered.map(extract_values)

In [ ]:
geemap.zonal_stats

In [ ]:
extracted.getInfo()

In [ ]:
df = pd.DataFrame(data["features"])
df["date"] = pd.to_datetime(
    df["properties"].apply(lambda x: x["system:time_start"]), unit="ms"
)
df["tmmx"] = df["properties"].apply(lambda x: x["tmmx"])
df = df[["date", "tmmx"]].set_index("date")